In [1]:
!tw-make tw-cooking --h

usage: tw-make tw-cooking [-h] [--recipe INT] [--take INT] [--go {1,6,9,12}]
                          [--open] [--cook] [--cut] [--drop]
                          [--recipe-seed INT] [--split {train,valid,test}]
                          [--output PATH] [--seed SEED] [--format {ulx,z8}]
                          [--overview] [--save-overview] [-f] [--silent | -v]

optional arguments:
  -h, --help            show this help message and exit

First TextWorld Competition game settings:
  --recipe INT          Number of ingredients in the recipe. Default: 1
  --take INT            Number of ingredients to find. It must be less or
                        equal to the value of `--recipe`. Default: 0
  --go {1,6,9,12}       Number of locations in the game (1, 6, 9, or 12).
                        Default: 1
  --open                Whether containers/doors need to be opened.
  --cook                Whether some ingredients need to be cooked.
  --cut                 Whether some ingredients nee

In [2]:
!tw-make tw-cooking --seed 5611

Global seed: 5611
Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/loa/bin/tw-make", line 211, in <module>
    game_file = textworld.generator.compile_game(game, options)
  File "/usr/local/Caskroom/miniconda/base/envs/loa/lib/python3.8/site-packages/textworld/generator/__init__.py", line 250, in compile_game
    assert already_compiled, msg
AssertionError: It's highly unprobable that two games with the same id have different structures. That would mean the generator has been modified. Please clean already generated games found in '/Users/mich/git/LOA-public/demo/tw_games'.


Customize Json2PDDL.py below.

`%load ../pddlgym/textworld/Json2PDDL.py`

In [3]:
!python ../pddlgym/textworld/Json2PDDL.py tw_games/tw-cooking-recipe1+go1-JrmLfNyMcErjF6LD.json

Traceback (most recent call last):
  File "../pddlgym/textworld/Json2PDDL.py", line 233, in <module>
    text_file.write(pddl_parser.get_pddl())
  File "../pddlgym/textworld/Json2PDDL.py", line 178, in get_pddl
    pddl_objects_string += self._list_to_pddl_format('objects', self.get_objects(), self.indentation_spaces)
  File "../pddlgym/textworld/Json2PDDL.py", line 55, in get_objects
    raise ValueError('Object type: ', type, ' is not handled')
ValueError: ('Object type: ', 'oven', ' is not handled')


In [11]:
# %load ../pddlgym/textworld/Json2PDDL.py
"""
Tools to read the TextWorld JSON game files into a PDDL format
"""
import json


class JsonToPddl:
    """ Class that takes care of different parsing portions to read the JSON file into the PDDL format
    """
    def __init__(self, json_file_path):
        self.json_file_path = json_file_path
        self.indentation_spaces = ' '*4  # convention for indentation of space/tabs in the PDDL file to write

        # This character replaces spaces while processing in the PDDL and planning portion
        self.space_string_proxy = '_'

        # Read the dictionary
        with open(self.json_file_path) as json_file:
            self.json_data = json.load(json_file)

        # Create a dictionary of the keys to the names of the TextWorld objects
        self.object_name_dict = {}
        for info in self.json_data['infos']:
            obj_name = info[1]['name']
            self.object_name_dict.update({info[0]: self._handle_spaces_in_name(obj_name)})

    def get_objects(self):
        """ Parse the object names from the data
        :return: list of strings each element being a pddl object
        """
        pddl_objects = []
        for info in self.json_data['infos']:
            type = info[1]['type']
            name = self._handle_spaces_in_name(info[1]['name'])
            if True:
                # no type name replacement
                pddl_objects.append(f'{name} - {type}')
            else:
                if type == 'r':
                    pddl_objects.append(name+' - room')
                elif (type == 'P') or (type == 'I'):
                    # P is the players/agent
                    # I is the players's inventory
                    # Both the player and inventory are assumed to always exist in the PDDL description
                    pass
                elif type == 'd':
                    pddl_objects.append(name+' - door')
                elif type == 'k':
                    pddl_objects.append(name + ' - key')
                elif type == 's':
                    pddl_objects.append(name + ' - supporter')
                elif type == 'c':
                    pddl_objects.append(name + ' - container')
                elif type == 'f':
                    pddl_objects.append(name + ' - food')
                elif type == 'o':
                    pddl_objects.append(name + ' - object')
                else:
                    raise ValueError('Object type: ', type, ' is not handled')
        return pddl_objects

    def get_init(self):
        """ Parse the object initializations from the data
        :return: list of strings each element being a pddl init
        """
        pddl_init = []
        world = self.json_data['world']

        # Always have the default door constant initialized as open
        pddl_init.append('(opened no-door)')

        # Pre-process the door names for the room links
        link_dict = {}
        for w in world:
            if w['name'] == 'link':
                w_args = w['arguments']
                link_dict.update({w_args[0]['name'] + w_args[2]['name']: w_args[1]['name']})

                # Handle the door location
                w_args[0], w_args[1] = w_args[1], w_args[0]
                pddl_init.append(self._create_predicate_triple('at', w_args))

        # Process all the variables
        for w in world:
            w_args = w['arguments']
            if w['name'] == 'at':
                if w_args[0]['type'] == 'P':
                    # Parse starting location of the agent
                    room_id = w_args[1]['name']
                    pddl_init.append('(at-agent ' + self.object_name_dict[room_id]+')')
                else:
                    # Parse location of objects in rooms
                    pddl_init.append(self._create_predicate_triple('at', w_args))
            elif w['name'] == 'in':
                if w_args[1]['name'] == 'I':
                    pddl_init.append('(carry ' + self.object_name_dict[w_args[0]['name']]+')')
                else:
                    pddl_init.append(self._create_predicate_triple('in', w_args))
            elif w['name'] == 'on':
                pddl_init.append(self._create_predicate_triple('on', w_args))
            # Parse the room connectivity
            elif w['name'] == 'north_of' or \
                 w['name'] == 'east_of' or \
                 w['name'] == 'south_of' or \
                 w['name'] == 'west_of':
                direction = w['name'].split('_')[0]
                room_1_key = w_args[0]['name']
                room_2_key = w_args[1]['name']
                link_key = room_1_key + room_2_key
                link_name = self.object_name_dict[link_dict[link_key]] if link_key in link_dict.keys() else 'no-door'
                pddl_init.append('(' +
                                 ' '.join(['room-connect',
                                           self.object_name_dict[room_1_key],
                                           direction,
                                           self.object_name_dict[room_2_key],
                                           link_name])
                                 + ')')
            elif w['name'] == 'open':
                pddl_init.append('(opened ' + self.object_name_dict[w_args[0]['name']] + ')')
            elif w['name'] == 'closed':
                pddl_init.append('(not (opened ' + self.object_name_dict[w_args[0]['name']] + '))')
            elif w['name'] == 'match':
                pddl_init.append(self._create_predicate_triple('keymatch', w_args))
            elif w['name'] == 'locked':
                pddl_init.append('(locked ' + self.object_name_dict[w_args[0]['name']] + ')')
            else:
                print(f"Unexpected {w['name']}")
                # These initializations are safe to ignore (handled elsewhere)
                if w['name'] not in ['free', 'link', 'edible']:
                    raise ValueError('Unknown TextWorld fact description: ', w)
        return pddl_init

    def get_goal(self):
        """ Parse the goals from the data
        :return: list of strings each element being a pddl goal
        """
        pddl_goal = []
        quest = self.json_data['main_quest'] if 'main_quest' in self.json_data else self.json_data['quests'][0]
        conds = quest['win_events'][0]['condition']['postconditions']
        for c in conds:
            c_args = c['arguments']
            if c['name'] == 'in':
                if c_args[1]['name'] == 'I':
                    pddl_goal.append('(carry ' + self.object_name_dict[c_args[0]['name']]+')')
                else:
                    pddl_goal.append(self._create_predicate_triple('in', c_args))
            elif c['name'] == 'on':
                pddl_goal.append(self._create_predicate_triple('on', c_args))
            elif c['name'] == 'at':
                if c_args[0]['name'] == 'P':
                    pddl_goal.append('(at-agent ' + self.object_name_dict[c_args[1]['name']] + ')')
                else:
                    pddl_goal.append(self._create_predicate_triple('at', c_args))
            elif c['name'] == 'open':
                pddl_goal.append('(opened ' + self.object_name_dict[c_args[0]['name']] + ')')
                pddl_goal.append('(not(locked ' + self.object_name_dict[c_args[0]['name']] + '))')
            elif c['name'] == 'closed':
                pddl_goal.append('(not (opened ' + self.object_name_dict[c_args[0]['name']] + '))')
                pddl_goal.append('(not (locked ' + self.object_name_dict[c_args[0]['name']] + '))')
            elif c['name'] == 'locked':
                pddl_goal.append('(locked ' + self.object_name_dict[c_args[0]['name']] + ')')
            elif c['name'] == 'eaten':
                pddl_goal.append('(eaten ' + self.object_name_dict[c_args[0]['name']] + ')')
            else:
                if c['name'] not in ['match', 'event', 'link', 'free']:
                    raise ValueError('Unknown TextWorld goal description: ', c)
        combined_pddl_goal = '(and'
        for g in pddl_goal:
            combined_pddl_goal += g
        combined_pddl_goal += ')'
        return [combined_pddl_goal]

    def get_pddl(self, problem_name='tw_game'):
        """ Get the full pddl string
        :param problem_name: name of the problem to be written
        :return: String in the PDDL format
        """
        # Header
        pddl_objects_string = '(define (problem ' + problem_name + ')\n'
        pddl_objects_string += self.indentation_spaces + '(:domain textworld)\n\n'

        # Main body - objects, init, goal
        pddl_objects_string += self._list_to_pddl_format('objects', self.get_objects(), self.indentation_spaces)
        pddl_objects_string += self._list_to_pddl_format('init', self.get_init(), self.indentation_spaces)
        pddl_objects_string += self._list_to_pddl_format('goal', self.get_goal(), self.indentation_spaces)

        # End
        pddl_objects_string += ')'
        return pddl_objects_string

    def _create_predicate_triple(self, predicate_name, game_logic_args):
        """ Helper function to handle parsing predicate triples
        :param predicate_name:
        :param game_logic_args:
        :return: pddl predicate string
        """
        return '(' + \
               ' '.join([predicate_name,
                         self.object_name_dict[game_logic_args[0]['name']],
                         self.object_name_dict[game_logic_args[1]['name']]]) \
               + ')'

    def _handle_spaces_in_name(self, name):
        """ Helper function to handle the spaces in the object names (for now translated into hyphens in pddl)
        :param name: object name in textworld
        :return: object name for pddl
        """
        return name.replace(' ', self.space_string_proxy) if name is not None else name

    def _list_to_pddl_format(self, header_name, pddl_list, spaces):
        """ Helper function to parse a list of things into the proper pddl syntax
        :param header_name: either 'objects', 'init' or 'goal'
        :param pddl_list:
        :param spaces: convention to use for spacing
        :return: string in pddl compatible syntax
        """
        pddl_formatted_string = spaces + '(:' + header_name + '\n'
        for x in pddl_list:
            pddl_formatted_string += spaces * 2 + x + '\n'
        pddl_formatted_string += spaces + ')\n\n'
        return pddl_formatted_string


# Simple test where the pddl is directly written to a file
if None and __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser(description='Create a PDDL problem file from a TextWorld JSON game file')
    parser.add_argument('textworld_json_file', type=str,
                        help='path to a json file of TextWorld')
    parser.add_argument('--output-filename', type=str,
                        default='output_problem.pddl',
                        help='output pddl problem filename')
    args = parser.parse_args()

    pddl_parser = JsonToPddl(args.textworld_json_file)
    text_file = open(args.output_filename, 'w')
    text_file.write(pddl_parser.get_pddl())
    text_file.close()


In [12]:
pddl_parser = JsonToPddl('tw_games/tw-cooking-recipe1+go1-JrmLfNyMcErjF6LD.json')
pddl_parser.get_pddl()

Unexpected base


ValueError: ('Unknown TextWorld fact description: ', {'name': 'base', 'arguments': [{'name': 'f_0', 'type': 'f'}, {'name': 'ingredient_0', 'type': 'ingredient'}]})